# FRAGMENT-MNP

FRAGMENT-MNP is a mechanistic model of Micro and NanoPlastic FRAGMentation in the ENvironmenT.
- [Project webpage](https://www.ceh.ac.uk/our-science/projects/fragment-mnp)
- [Model code repository](https://github.com/microplastics-cluster/fragment-mnp)
- [Model documentation](https://microplastics-cluster.github.io/fragment-mnp)

This app is a simple demonstration of the latest version of the model. The model is still under active development and so except frequent changes, including additions to the model conceptualisation.

## Theory

FRAGMENT-MNP models the time evolution of micro and nanoplastic particle number concentrations by splitting the size distribution of plastic particles into a number of size classes (bins) and numerically solving the following differential equation for each size class:

$$
\frac{dn_k}{dt} = -k_{\text{frag},k} n_k + \Sigma_i f_{i,k} k_{\text{frag},i} n_i - k_\text{diss} n_k
$$

Here, $n_k$ is the particle number concentration in size class $k$, $t$ is time, $k_{\text{frag},k}$ and $k_{\text{diss},k}$ are the fragmentation and dissolution rates of size class $k$, and $f_{i,k}$ is the fraction of daughter fragments produced from a fragmenting particle of size $i$ that are of size $k$. Note that there is no source term - the only input of plastic particles is the initial value at $t=0$.

$k_\text{frag}$ and $k_\text{diss}$ are implicitly dependent on the phys-chem properties and the environment the polymer is in (which governs the degradation and mechanical stresses it encounters). For $k_\text{frag}$, we express this using energy dissipation rate $\epsilon$ and surface energy $\sigma$, along with empirical constants $\Phi$ and $\theta$:

$$
k_\text{frag} = \Phi_1 \sigma^{-\theta_1} \Phi_2 \epsilon^{\theta_2}
$$

The surface energy is inversely proportional to the square of the diameter of the particle, $\sigma \propto d_p^{-2}$, and so for an energy dissipation rate that is constant at each length scale, $k_\text{frag}$ varies as $d_p^{2\theta_1}$. We use this to vary $k_\text{frag}$ across size classes using a given average $k_\text{frag}$ and value for $\theta_1$. We also use this to scale $k_\text{diss}$ as proportional to the surface-to-volume ratio of the particles $s$, and use another proportionality constant $\gamma$ to do so: $k_\text{diss} \propto s^\gamma$. To convert particle number to mass concentrations, we assume spherical particles.



In [2]:
import numpy as np
from ipywidgets import widgets, interact
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import cm
from matplotlib.colors import rgb2hex
from fragmentmnp import FragmentMNP
from fragmentmnp.examples import minimal_config, minimal_data

style = {'description_width': '250px'}
layout = widgets.Layout(width='600px')

@interact
def fragmentmnp(K=widgets.IntSlider(min=1, max=20, value=7, description="Number of size classes", style=style, layout=layout),
                K_range=widgets.IntRangeSlider(min=-15, max=-2, value=[-9, -3], description="Size class diameter range (10^value)", style=style, layout=layout),
                T=widgets.IntSlider(min=1, max=1000, value=100, description="Number of timesteps", style=style, layout=layout),
                bar_ts=widgets.IntText(value=99, description="Show bar chart for time step...", style=style, layout=layout),
                _k_frag=widgets.BoundedFloatText(0.01, min=0.0, max=1.0, step=0.01, description="Average fragmentation rate", style=style, layout=layout),
                theta1=widgets.BoundedFloatText(0.0, min=0.0, max=0.5, step=0.01, description="Empirical constant theta1", style=style, layout=layout),
                k_diss=widgets.BoundedFloatText(0.0005, min=0.0, max=0.1, step=0.0001, description="Average dissolution rate", style=style, layout=layout),
                gamma1=widgets.BoundedFloatText(1, min=0.0, max=5, step=0.1, description="Empirical constant gamma1", style=style, layout=layout),
                _n_0=widgets.FloatText(42.0, description="Initial particle number concentration", style=style, layout=layout),
                density=widgets.BoundedFloatText(1380, min=1, max=1e5, step=10, description='Polymer density', style=style, layout=layout)):

    # Use the parameters provided to edit the config and data
    minimal_config['n_size_classes'] = K
    minimal_config['particle_size_range'] = K_range
    minimal_config['n_timesteps'] = T
    minimal_config['k_diss_scaling_method'] = 'surface_area'
    minimal_data['initial_concs'] = [_n_0] * K
    minimal_data['density'] = density
    minimal_data['k_frag'] = _k_frag
    minimal_data['theta_1'] = theta1
    minimal_data['k_diss'] = k_diss
    minimal_data['k_diss_gamma'] = gamma1

    # Create and run the model
    fmnp = FragmentMNP(minimal_config, minimal_data)
    output = fmnp.run()

    # Define the colour map to use
    viridis = [rgb2hex(rgb) for rgb in cm.get_cmap('viridis', K).colors]
  
    # Set up the subplots
    fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.12,
                        subplot_titles=('', f'Time step: {bar_ts}'),
                        specs=[[{"secondary_y": True},{}]])

    # Line chart timeseries first
    for i in range(0, K):
        fig.add_trace(go.Scatter(x=output.t, y=output.n[i], name=f'{fmnp.psd[i]:.2e} m',
                      line_color=viridis[i]), row=1, col=1)
    # # Plot the loss (if there is any) with a different style
    fig.add_trace(go.Scatter(x=output.t, y=np.sum(output.c_diss, axis=0), name='Dissolution',
                                line={'width': 3, 'dash': 'dash', 'color': 'lightblue'}), row=1, col=1, secondary_y=True)
    fig.update_xaxes(title='Time', col=1, row=1)
    fig.update_yaxes(title='Particle number conc [particles/volume]', col=1, row=1)
    fig.update_yaxes(title='Dissolution mass conc [mass/volume]', col=1, row=1, showgrid=False, secondary_y=True)
  
    # Now the bar chart
    if bar_ts < T:
        fig.append_trace(go.Bar(x=[f'{sc:.2e} m' for sc in fmnp.psd], y=output.n[:, bar_ts],
                                marker_color=viridis, showlegend=False), row=1, col=2)
    fig.update_xaxes(title='Size class diameter', col=2, row=1)
    fig.update_yaxes(title='Particle number conc [particles/volume]', col=2, row=1, showticklabels=True)

    # Update the layout and show
    fig.update_layout(width=1300, legend={'yanchor': 'top', 'y': 0.99, 'xanchor': 'left', 'x': 0.96})
    fig.show()


interactive(children=(IntSlider(value=7, description='Number of size classes', layout=Layout(width='600px'), m…